In [86]:
#라이브러리 임포트
import torch
import torch.nn as nn   # 신경망 모듈
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [87]:
import requests
dataset_file_origin = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

# 요청을 보내고 응답을 받기
response = requests.get(dataset_file_origin)

# 응답이 성공적이면 파일로 저장
if response.status_code == 200:
    # with 문 : 파일을 열고 작업을 완료한 후에 자동으로 파일을 닫아준다
    with open('shakespeare.txt', 'w', encoding="utf-8") as file:
        file.write(response.text)
        
    print("파일이 성공적으로 다운로드되었습니다.")
    
else:
    print(f"파일 다운로드 실패: 상태 코드 {response.status_code}")
        

파일이 성공적으로 다운로드되었습니다.


In [88]:
text=""
with open("shakespeare.txt", "r", encoding="utf-8") as file:
    text = file.read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [89]:
# 데이터 전처리

# 문자 단위로 잘라서 리스트로 생성
'''
문자열을 단어 단위가 아니라 문자 단위로 전처리하는 이유?

- 단어 단위보다 문자 단위로 작업하면 모델이 보다 세밀한 텍스트를 생성할 수 있다
- 모델 복잡도를 줄여준다. 특정 언어에 종속되지 않는다.
    근데 여기서 의문점... 한국어는 어떻게 처리할까?
     => 한 글자 단위로 분해하게 되는데, 자음과 모음의 조합으로 음절을 형성함으로, 조합 가능한 개수가 더 많다. (영어는 알파벳, 숫자, 구두점 등을 모두 포함해도 100미만의 문자로 충분)
     => 따라서, 한국어를 문자 단위로 처리할 떄는 더 많은 유니크한 문자를 처리해야 하는데, 이러한 접근 방식은 한국의 음절 구조와 자모 결합 패턴을 학습할 수 있다는 것이다.
'''
chars = sorted(list(set(text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

In [90]:
# 시퀀스 데이터 생성 함수 정의
'''
텍스트 데이터를 일정 길이의 시퀀스로 분할하고, 시퀀스 다음에 오는 문자를 타겟으로 설정하여
시퀀스-타켓 쌍을 생성하는 함수
'''
def creat_sequences(text, seq_length):
 sequences = []
 targets = []
 for i in range(0, len(text) - seq_length):
     # 시퀀스 생성
     seq = text[i:i+seq_length]
     # 시퀀스 다음에 오는 문자
     target = text[i+seq_length]
     sequences.append([char_to_idx[char] for char in seq])
     targets.append(char_to_idx[target])
 return sequences, targets
         

In [91]:
# 시퀀스 길이 설정
seq_length = 100

# 시퀀스 데이터 생성
sequences, targets = creat_sequences(text, seq_length)

In [92]:
# 문자를 고유한 정수 인덱스로 매핑한 값들 출력
print(sequences[:10])

[[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59], [47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59, 1], [56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50

In [93]:
# Pytorch 데이터셋 및 데이터로더 생성

# Pytorch의 Dataset 상속 받은 TextDataset 클래스 정의
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.targets[idx], dtype=torch.long)

In [94]:
# 데이터셋 및 데이터로더 인스턴스 생성
dataset = TextDataset(sequences, targets)

# 데이터로더 : 데이터셋을 배치 단위로 묶어서 전달해주는 역할
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
'''
<batch_size = 64>
- 매번 64개의 시퀀스-타켓 쌍을 한 배치로 묶어서 모델에 제공
- 예를 들어, 데이터셋에 1000개의 시퀀스가 있으면, 데이터로더는 1000/64 = 16개의 배치를 생성해 16번 배치를 모델에 전달함

<shuffle = True>
- 데이터를 섞어서 모델이 학습할 때 순서를 학습하는 것을 방지
'''

'\n<batch_size = 64>\n- 매번 64개의 시퀀스-타켓 쌍을 한 배치로 묶어서 모델에 제공\n- 예를 들어, 데이터셋에 1000개의 시퀀스가 있으면, 데이터로더는 1000/64 = 16개의 배치를 생성해 16번 배치를 모델에 전달함\n\n<shuffle = True>\n- 데이터를 섞어서 모델이 학습할 때 순서를 학습하는 것을 방지\n'

In [95]:
# 하이퍼파라미터 설정
vocab_size = len(chars)
hidden_size = 256
output_size = len(chars)
num_layers = 2

In [96]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
'''
device 변수
: PyTorch에서 모델과 데이터를 GPU 또는 CPU에 올리기 위해 사용
=> GPU가 사용 가능하면, cuda로 설정하고, 그렇지 않으면 cpu로 설정
'''

'\ndevice 변수\n: PyTorch에서 모델과 데이터를 GPU 또는 CPU에 올리기 위해 사용\n=> GPU가 사용 가능하면, cuda로 설정하고, 그렇지 않으면 cpu로 설정\n'

In [97]:
# RNN 모델을 사용한 클래스 정의
class RNNModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(vocab_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

In [98]:
# 모델 인스턴스 생성 및 GPU로 이동
model = RNNModel(vocab_size, hidden_size, output_size, num_layers).to(device)

In [99]:
# 손실 함수와 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [100]:
# 모델 훈련 함수
def train_model(model, dataloader, criterion, optimizer, num_epochs=200):
    # 모델을 훈련 모드로 설정
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            # 원-핫 인코딩 및 GPU로 이동
            inputs = nn.functional.one_hot(inputs, num_classes=vocab_size).float().to(device)
            labels = labels.to(device)
            
            # 각 배치마다 hidden 상태 초기화
            hidden = model.init_hidden(inputs.size(0))
            
            # 옵티마이저 초기화
            optimizer.zero_grad()
            
            # 순전파, 역전파, 최적화
            outputs, hidden = model(inputs, hidden)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # hidden 상태를 detach()로 분리
            hidden = hidden.detach()
            
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}")
    print('Finished Training')

In [101]:
# 텍스트 생성 함수
def gererate_text(model, start_str, length, temperature=1.0):
    # 모델을 평가 모드로 설정
    model.eval()
    hidden = model.init_hidden(1)
    
    input_seq = torch.tensor([char_to_idx[char] for char in start_str]).unsqueeze(0).to(device)
    generated_text = start_str
    
    with torch.no_grad():
        for _ in range(length):
            input_seq = nn.functional.one_hot(input_seq, num_classes=vocab_size).float()
            output, hidden = model(input_seq, hidden)
            
            # 다음 문자를 샘플링
            output = output.squeeze().div(temperature).exp().cpu()
            top_char = torch.multinomial(output, 1)[0]
            
            generated_char = idx_to_char[top_char.item()]
            generated_text += generated_char
            
            # 다음 입력 시퀀스 준비
            input_seq = torch.tensor([[top_char]]).to(device)
    
    return generated_text

In [102]:
print(model)

RNNModel(
  (rnn): RNN(65, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=65, bias=True)
)


In [103]:
# 모델 훈련
train_model(model, dataloader, criterion, optimizer, num_epochs=5)

KeyboardInterrupt: 

In [ ]:
# 테스트 시작 문자열 및 생서할 텍스트 길이 설정
start_str = 'To be, or not to be, that is the question:'
length = 200

In [ ]:
# 텍스트 생성
generated_text = gererate_text(model, start_str, length, temperature=0.8)
print(generated_text)